# Staatsangehörigkeit
- Karte Heat
- Karte Bubbles

In [1]:
import plotly.io as pio

# Benutzerdefiniertes Template definieren
infoviz_template = dict(
    layout=dict(
        template="plotly_white",
        title=dict(
            font=dict(size=20, family="Arial", weight="bold", color="black"),
            y=0.91,  # Titel weiter nach oben
            x=0.05,  # Links ausgerichtet
            xanchor="left",  # Linksbündig
        ),
        xaxis=dict(
            showgrid=False,
            zerolinecolor="lightgrey",
            tickfont=dict(color="grey", size=12),
            title_font=dict(color="grey", weight="bold", size=13),
            title_standoff=15,
            ticklabelposition="outside bottom"
        ),
        yaxis=dict(
            showgrid=True, gridcolor="lightgrey",
            zerolinecolor="lightgrey",
            tickfont=dict(color="grey", size=12),
            title_font=dict(color="grey", weight="bold", size=13),
            title_standoff=15,
            ticklabelposition="outside left"
        ),
    )
)
pio.templates["infoviz"] = infoviz_template

In [2]:
import pandas as pd
import plotly.express as px

# Datei einlesen (ersetze 'data.xlsx' durch den tatsächlichen Dateinamen)
file_path = "data.xlsx"

# Alle Sheets einlesen, aber das erste Blatt ignorieren
sheets = pd.read_excel(file_path, sheet_name=None, engine="openpyxl")

# Erstes Blatt entfernen
sheets.pop(next(iter(sheets)))  

# Dictionary zur Speicherung der Werte
staatsangehoerigkeit = {"Jahr": [], "Deutsche Staatsangehörigkeit (%)": [], "Andere Staatsangehörigkeit (%)": []}

# Durch alle Jahre iterieren
for jahr, df in sheets.items():
    df.columns = df.columns.astype(str).str.strip()  # Saubere Spaltennamen
    
    # Extrahiere die Zeilen für Staatsangehörigkeit
    deutsch_row = df[df["Variable"].str.contains("Staatsangehörigkeit", na=False) & 
                     df["Category"].str.contains("Deutsche Staatsangehörigkeit", na=False)]
    
    andere_row = df[df["Variable"].str.contains("Staatsangehörigkeit", na=False) & 
                    df["Category"].str.contains("Andere Staatsangehörigkeit", na=False)]
    
    if not deutsch_row.empty and not andere_row.empty:
        # Werte speichern
        staatsangehoerigkeit["Jahr"].append(jahr)
        staatsangehoerigkeit["Deutsche Staatsangehörigkeit (%)"].append(deutsch_row.iloc[:, 2:].mean(axis=1).values[0])
        staatsangehoerigkeit["Andere Staatsangehörigkeit (%)"].append(andere_row.iloc[:, 2:].mean(axis=1).values[0])

# Daten als DataFrame formatieren
df_staatsangehoerigkeit = pd.DataFrame(staatsangehoerigkeit).sort_values(by="Jahr")

In [4]:
fig = px.area(
    df_staatsangehoerigkeit, 
    x="Jahr", 
    y=["Deutsche Staatsangehörigkeit (%)", "Andere Staatsangehörigkeit (%)"], 
    title="Entwicklung der Staatsangehörigkeit von Studierenden",
    labels={"value": "Anteil (%)", "variable": "Staatsangehörigkeit"},
    color_discrete_sequence=px.colors.qualitative.Safe,
    template="infoviz"
)

fig.update_layout(
    template="infoviz",
    xaxis_title="Jahr",
    yaxis_title="Anteil der Studierenden (%)",
    yaxis=dict(range=[0, 100]),  # Da die Summe 100% ergibt
    legend_title="Kategorie"
)

fig.show()

In [5]:
# Daten umformen für Balkendiagramm
df_melted = df_staatsangehoerigkeit.melt(id_vars="Jahr", var_name="Staatsangehörigkeit", value_name="Anteil (%)")

fig = px.bar(
    df_melted, 
    x="Jahr", 
    y="Anteil (%)", 
    color="Staatsangehörigkeit",
    barmode="group",  # Gruppierte Balken
    title="Entwicklung der Staatsangehörigkeit von Studierenden",
    color_discrete_sequence=px.colors.qualitative.Safe
)

fig.update_layout(
    template="infoviz",
    xaxis_title="Jahr",
    yaxis_title="Anteil der Studierenden (%)",
    yaxis=dict(range=[0, 100]),
    legend_title="Kategorie"
)

fig.show()

## Bachelor vs Master

In [6]:
import pandas as pd
import plotly.express as px

# Datei einlesen (ersetze 'data.xlsx' durch den tatsächlichen Dateinamen)
file_path = "data.xlsx"

# Alle Sheets einlesen, aber das erste Blatt ignorieren
sheets = pd.read_excel(file_path, sheet_name=None, engine="openpyxl")

# Erstes Blatt entfernen
sheets.pop(next(iter(sheets)))  

# Dictionary zur Speicherung der Werte
staatsangehoerigkeit = {"Jahr": [], "Bachelor (%)": [], "Master (%)": []}

# Durch alle Jahre iterieren
for jahr, df in sheets.items():
    df.columns = df.columns.astype(str).str.strip()  # Spaltennamen bereinigen
    
    # Studiengänge automatisch erkennen
    bachelor_studiengaenge = [col for col in df.columns if "Bachelor" in col]
    master_studiengaenge = [col for col in df.columns if "Master" in col]

    # Extrahiere die Zeilen für "Andere Staatsangehörigkeit (%)"
    andere_row = df[df["Variable"].str.contains("Staatsangehörigkeit", na=False) & 
                    df["Category"].str.contains("Andere Staatsangehörigkeit", na=False)]
    
    if not andere_row.empty:
        # Berechnung des gewichteten Anteils für Bachelor & Master
        bachelor_mean = andere_row[bachelor_studiengaenge].mean(axis=1).values[0]
        master_mean = andere_row[master_studiengaenge].mean(axis=1).values[0]

        staatsangehoerigkeit["Jahr"].append(jahr)
        staatsangehoerigkeit["Bachelor (%)"].append(bachelor_mean)
        staatsangehoerigkeit["Master (%)"].append(master_mean)

# Daten als DataFrame formatieren
df_staatsangehoerigkeit = pd.DataFrame(staatsangehoerigkeit).sort_values(by="Jahr")

# DataFrame für Plotly umstrukturieren
df_melted = df_staatsangehoerigkeit.melt(id_vars="Jahr", var_name="Studienabschluss", value_name="Anteil Andere Staatsangehörigkeit (%)")

In [7]:
# Farben für Bachelor & Master definieren
color_map = {
    "Bachelor (%)": "#1f77b4",  # Blau
    "Master (%)": "#d62728"  # Rot
}

# Liniendiagramm mit Plotly erstellen
fig = px.line(
    df_melted,
    x="Jahr",
    y="Anteil Andere Staatsangehörigkeit (%)",
    color="Studienabschluss",
    markers=True,
    title="Anteil Studierender mit anderer Staatsangehörigkeit (Bachelor vs. Master)",
    color_discrete_map=color_map,
    template="infoviz"
)

# Layout-Anpassungen
fig.update_traces(line=dict(width=3))  # Dickere Linien
fig.update_layout(
    xaxis=dict(title="Jahr"),
    yaxis=dict(title="Anteil Andere Staatsangehörigkeit (%)"),
    legend_title="Studienabschluss"
)

fig.show()